<a href="https://colab.research.google.com/github/FlyAIBox/langchain-academy/blob/fly101/module-6/creating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 创建部署

我们将为在第 5 模块构建的 `task_assistant ` 应用创建一次部署。部署（deployment）指的是把已经实现的 LangGraph 应用和它所依赖的资源打包，并让它能够在指定环境中稳定运行。

## 代码结构

创建 LangGraph 平台部署时需要准备以下内容（参见链接获取更详细的结构说明）：

* LangGraph API 配置文件 `langgraph.json`
* 实现应用逻辑的图脚本，例如 `task_assistant .py`
* 列出运行所需依赖的文件 `requirements.txt`
* 提供运行所需环境变量的配置文件，如复制`.env-example` 为`.env` 或 `docker-compose.yml`

这些文件已经放在 `module-6/deployment` 目录中，可以直接使用。

## CLI

[LangGraph CLI](https://langchain-ai.github.io/langgraph/concepts/langgraph_cli/) 是用于创建 LangGraph 平台部署的命令行工具（Command-Line Interface）。


In [ ]:
%%capture --no-stderr
%pip install -U langgraph-cli

在创建[自托管部署](https://langchain-ai.github.io/langgraph/how-tos/deploy-self-hosted/#how-to-do-a-self-hosted-deployment-of-langgraph)时，我们按照以下步骤操作。

### 构建 LangGraph Server 的 Docker 镜像

首先使用 LangGraph CLI 为 [LangGraph Server](https://docs.google.com/presentation/d/18MwIaNR2m4Oba6roK_2VQcBE_8Jq_SI7VHTXJdl7raU/edit#slide=id.g313fb160676_0_32) 构建一个 Docker 镜像。这个命令会把我们的图（graph）代码和 `requirements.txt` 中列出的依赖一起打包到镜像里。Docker 镜像可以理解为创建容器时的蓝图，里面包含了启动应用所需的全部代码和环境。

请确认已经安装好 [Docker](https://docs.docker.com/engine/install/)，然后运行下列命令生成名为 `my-image` 的镜像：

```
$ cd module-6/deployment
$ langgraph build -t my-image
```

### 配置 Redis 和 PostgreSQL

如果你已经在本地或其他服务器上运行了 Redis 和 PostgreSQL，可以只启动 LangGraph Server 容器，并将 Redis 与数据库的连接地址通过环境变量传入：

```
docker run \
    --env-file .env \
    -p 8123:8000 \
    -e REDIS_URI="foo" \
    -e DATABASE_URI="bar" \
    -e LANGSMITH_API_KEY="baz" \
    my-image
```

其中 `REDIS_URI` 和 `DATABASE_URI` 分别指向你的 Redis 与 Postgres 服务地址，`LANGSMITH_API_KEY` 用于让应用在 LangSmith 上记录运行数据。

如果你想一次启动所有组件，可以复制 `docker-compose-example.yml`，再补充下面这些环境变量，就能通过 Docker Compose 同时创建对应的三个容器：

* `IMAGE_NAME`（例如 `my-image`，即上一步构建的镜像名称）
* `LANGSMITH_API_KEY`
* `OPENAI_API_KEY`

准备好 `docker-compose.yml` 后，执行以下命令即可[启动部署](https://langchain-ai.github.io/langgraph/how-tos/deploy-self-hosted/#using-docker-compose)：

```
$ cd module-6/deployment
$ docker compose up
```

运行成功后，LangGraph Server、Redis 与 PostgreSQL 会分别在独立的容器中启动，便于本地调试或部署到服务器。
